In [ ]:
import csv
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline, make_lsq_spline
from scipy.optimize import curve_fit

# Colormaps: paired
# Extract colors from colormap, to use at will
colormap = cm['Paired']
colors = [colormap(i) for i in np.linspace(0, 1, 12)]

# Define fitting function
#def fitting_exponential(x, y0, v0, k):
#    return y0 - (v0/k) * (1 - np.exp(-k*x))

def fitting_exponential(x, a, c, d):
    return a*np.exp(-c*x)+d

def fitting_sigmoid(x, a, b, c, d):
    return d + (a - d)/(1 + np.power(x / c, b))

# Wrappers for easier use of interpolate functions from scipy
def generate_interp_spline(time, product, time_smooth):
    time = time[:len(product)]
    spl = make_interp_spline(time, product, k=3)  # type: BSpline
    smooth_data = spl(time_smooth)
    return smooth_data

def generate_lsq_spline(time, product, time_smooth):
    time = time[:len(product)]
    popt, pcov = curve_fit(fitting_exponential, time, product)
    print(popt)
    smooth_data = fitting_exponential(time_smooth, *popt)
    return smooth_data

In [ ]:
PNO_time = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", []), ("Re-ref", [])])
PNO_yield = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", []), ("Re-ref", [])])

with open('./data/PNO_Optimisation.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    next(plots)
    next(plots)
    for row in plots:
        PNO_time["5%_noH2O"].append(row[0])
        PNO_yield["5%_noH2O"].append(row[1])
        PNO_time["1%"].append(row[2])
        PNO_yield["1%"].append(row[3])
        PNO_time["2%"].append(row[4])
        PNO_yield["2%"].append(row[5])
        PNO_time["5%"].append(row[6])
        PNO_yield["5%"].append(row[7])
        PNO_time["Re-ref"].append(row[8])
        PNO_yield["Re-ref"].append(row[9])

# Cast str to float
PNO_yield = {j:[float(i) for i in PNO_yield[j] if i] for j in PNO_yield}
PNO_time = {j:[float(i) for i in PNO_time[j] if i] for j in PNO_time}

# Smoothen data for fitting
time_smooth = np.linspace(0, 48, 300)
smooth_data = dict([("1%", []), ("2%", []), ("5%", []), ("5%_noH2O", []), ("Re-ref", [])])

# Fittings from origin, Model ExpGro1
smooth_data["1%"] = [81.35845*np.exp(-4.78108/(x+1.22669)) for x in time_smooth]
smooth_data["2%"] = [np.exp(4.36918 + -3.27342/(x+0.9801)) for x in time_smooth]
smooth_data["5%"] = [-84.96408*np.exp(-x/4.97392) + 84.96408 for x in time_smooth]
smooth_data["5%_noH2O"] = [-47.14067*np.exp(-x/9.29703) + 47.14067 for x in time_smooth]
smooth_data["Re-ref"] = [ -23.11424*np.exp(-x/12.52357) -46.15156*np.exp(-x/202.73921) + 69.29016 for x in time_smooth]


In [ ]:
# Setup axes
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(0, 35)
axis.set_ylim(-5, 100)
axis.set_ylabel("Yield (%)")

# Plot lines

plt.plot(PNO_time['1%'], PNO_yield['1%'],'o' , zorder=2.6, color=colors[5], label="$\\bf{1}$, 1 mol%")
plt.plot(time_smooth, smooth_data['1%'],'-' , zorder=1.6, color=colors[4])
plt.plot(PNO_time['2%'], PNO_yield['2%'],'o' , zorder=2.6, color=colors[1], label="$\\bf{1}$, 2 mol%")
plt.plot(time_smooth, smooth_data['2%'],'-' , zorder=1.6, color=colors[0])
plt.plot(PNO_time['5%'], PNO_yield['5%'],'o' , zorder=2.6, color=colors[3], label="$\\bf{1}$, 5 mol%")
plt.plot(time_smooth, smooth_data['5%'],'-' , zorder=1.6, color=colors[2])
plt.plot(PNO_time['5%_noH2O'], PNO_yield['5%_noH2O'],'o' , zorder=2.6, color=colors[7], label="$\\bf{1}$, 5 mol%, no H$_2$O")
plt.plot(time_smooth, smooth_data['5%_noH2O'],'-' , zorder=1.6, color=colors[6])
plt.plot(PNO_time['Re-ref'], PNO_yield['Re-ref'],'o' , zorder=2.6, color=colors[9], label="$\\bf{1}$, Cool white LED (8 W)")
plt.plot(time_smooth, smooth_data['Re-ref'],'-' , zorder=1.6, color=colors[8])

plt.legend(fontsize='small')
plt.tight_layout()
# Save file
plt.savefig("./data/PNO_Optimization.jpg", dpi=300)

In [ ]:
time = list()
yield_PNO = dict([("3cm1", []), ("3cm2", []), ("6cm1", []), ("6cm2", [])])

with open('./data/Photochem_Setup_Validation.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    for row in plots:
        time.append(row[0])
        yield_PNO["3cm1"].append(row[1])
        yield_PNO["3cm2"].append(row[2])
        yield_PNO["6cm1"].append(row[3])
        yield_PNO["6cm2"].append(row[4])


# Cast str to float
yield_PNO = {j:[float(i) if i else np.nan for i in yield_PNO[j] ] for j in yield_PNO}
time = [float(i) for i in time]

# Smoothen data for fitting
time_smooth = np.linspace(0, 150, 300)
smooth_data = dict([("3cm1", []), ("3cm2", []), ("6cm1", []), ("6cm2", [])])

# Fittings form origin, Model ExpGro1
smooth_data["3cm1"] = [29.66303 -30.87762 * np.exp(-(x+1.09072)/18.1435) for x in time_smooth]
smooth_data["3cm2"] = [3.14379e14 * (1 - np.exp(-x/2.5462e15)) + 19.09884*(1 - np.exp(-x/4.63879)) for x in time_smooth]
smooth_data["6cm1"] = [16.9235 * (1 - np.exp(-x/8.54376)) + 25.96099*(1 - np.exp(-x/380.84198)) for x in time_smooth]
smooth_data["6cm2"] = [8.96422 * (1 - np.exp(-x/0.378)) + 21.74154*(1 - np.exp(-x/67.86183)) for x in time_smooth]

smooth_data["3cm"] = [ 34.63805 * x / (14.08940 + x) for x in time_smooth]
smooth_data["6cm"] = [ 26.33697 * x / (12.57343 + x) for x in time_smooth]


In [ ]:
# Setup axes
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(-5, 150)
axis.set_ylim(-5, 60)
axis.set_ylabel("Yield (%)")

# Plot lines
np.shape(time)
np.shape(yield_PNO)

plt.plot(time, yield_PNO['3cm1'], 'o', zorder=2.6, color=colors[5], label="3 cm")
plt.plot(time, yield_PNO['3cm2'], 'o', zorder=2.6, color=colors[1], label="3 cm")
plt.plot(time, yield_PNO['6cm1'], 'o', zorder=2.6, color=colors[3], label="6 cm")
plt.plot(time, yield_PNO['6cm2'], 'o', zorder=2.6, color=colors[7], label="6 cm")
#plt.plot(time_smooth, smooth_data['3cm1'], '-', zorder=1.6, color=colors[4])
#plt.plot(time_smooth, smooth_data['3cm2'], '-', zorder=1.6, color=colors[0])
#plt.plot(time_smooth, smooth_data['6cm1'], '-', zorder=1.6, color=colors[2])
#plt.plot(time_smooth, smooth_data['6cm2'], '-', zorder=1.6, color=colors[6])

plt.legend(fontsize='small')
plt.tight_layout()
# Save file
plt.savefig("./data/Photochem_Setup_Validation.jpg", dpi=300)


In [ ]:
time = list()
yield_PNO = dict([("H2O", []), ("D2O", [])])

with open('./data/PNO_KIEdata.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    for row in plots:
        time.append(row[0])
        yield_PNO["H2O"].append(row[1])
        yield_PNO["D2O"].append(row[2])

# Cast str to float
yield_PNO = {j:[float(i) if i else np.nan for i in yield_PNO[j] ] for j in yield_PNO}
time = [float(i) for i in time]

# Smoothen data for fitting
time_smooth = np.linspace(0, 210, 300)
smooth_data = dict([("H2O", []), ("D2O", [])])

# Fittings form origin, Model Polynomial fitting for H2O, expgro1 for D2O
smooth_data["H2O"] = [0.34785*x -0.00144 * x**2 + 2.35405e-6 * x**3  for x in time_smooth]
smooth_data["D2O"] = [27.56562 - 27.56731 * np.exp(x / -98.32344) for x in time_smooth]


In [ ]:
# Setup axes
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Reaction time (s)")
axis.set_xlim(-5, 200)
axis.set_ylim(-2, 35)
axis.set_ylabel("NMR Yield (%)")

# Plot lines
np.shape(time)
np.shape(yield_PNO)

plt.plot(time, yield_PNO['H2O'], 'o', zorder=2.6, color=colors[3], label="H$_2$O (3 M)")
plt.plot(time, yield_PNO['D2O'], 'o', zorder=2.6, color=colors[1], label="D$_2$O (3 M)")
plt.plot(time_smooth, smooth_data['H2O'], '-', zorder=1.6, color=colors[2])
plt.plot(time_smooth, smooth_data['D2O'], '-', zorder=1.6, color=colors[0])

plt.legend(fontsize='small')
plt.tight_layout()
# Save file
plt.savefig("./data/PNO_KIE.png", dpi=300)
